# 조사 및 기사 작성 Agent 개발

멀티 에이전트 시스템 기본 개념 및 CrewAI 프레임워크 소개

In [11]:
%pip install -q crewai crewai_tools langchain langchain_community


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [13]:
from crewai import Agent, Task, Crew

In [16]:
import os
import getpass

# os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'
os.environ["OPENAI_API_KEY"] = getpass.getpass()


 ········


In [17]:
# llm = ChatOpenAI(model="gpt-4o-mini")

# Agent 생성

- 에이전트를 정의하고 `역할(role)`, `목표(goal)` 및 `배경 스토리(backstory)`를 제공합니다.
- LLM은 롤플레잉을 할 때 더 나은 성과를 거두는 것으로 나타났습니다.

### Agent: Planner

In [19]:
    planner = Agent(
        role="콘텐츠 기획자",
        goal="{topic}에 대한 흥미롭고 사실에 기반한 콘텐츠 기획",
        backstory="당신은 {topic}에 대한 블로그 기사를 기획하고 있습니다. "
                  "독자가 새로운 정보를 배우고,  "
                  "올바른 결정을 내리는 데 도움이 될 수 있는 자료를 수집합니다. "
                  "당신의 기획 내용은 콘텐츠 작성자가 이 주제에 대한 기사를 작성하는 "
                  "이 주제에 대한 기사를 작성하는 기반이 됩니다.",
        allow_delegation=False,
        verbose=True,
        llm="gpt-4o-mini"
    )

### Agent: Writer

In [21]:
writer = Agent(
    role="콘텐츠 작성자",
    goal="{topic} 에 대한 통찰력 있고 사실에 기반한 "
         "의견 기사를 한글로 작성하는 것",
    backstory="당신은 {topic} 에 대한 새로운 블로그 게시글을 작성하고 있습니다. "
              "이 기사는  콘텐츠 기획자가 제공하는 개요와 관련 정보를 기반으로 합니다. "
              "당신은 콘텐츠 기획자가 제공한 개요의 주요 목표와 방향을 따릅니다. "
              "또한, 객관적이고 공정한 통찰을 제공하며, "
              "이를 콘텐츠 기획자가 제공한 정보로 뒷받침합니다. "
              "블로그 게시글에서는 의견과 객관적 사실을 명확히 구분하여 서술합니다.",
    allow_delegation=False,
    verbose=True,
    llm="gpt-4o-mini"
)

### Agent: Editor

In [22]:
editor = Agent(
    role="편집자",
    goal="주어진 기사를 조직의 글쓰기 스타일에 맞게 편집합니다.",
    backstory="당신은 콘텐츠 작성자 로부터 블로그 게시글을 전달받는 편집자입니다. "
              "당신의 목표는 블로그 게시글을 검토하여 저널리즘의 최선의 관행을 따르도록 하고, "
              "의견이나 주장을 제공할 때 균형 잡힌 관점을 유지하며, "
              "가능하면 주요 논란이 되는 주제나 의견을 피하도록 하는 것입니다.",
    allow_delegation=False,
    verbose=True,
    llm="gpt-4o-mini"
)

## Task 생성

- Task를 정의하고 `설명(description)`, `예상 출력(expected_output)` 및 `에이전트(agent)`를 제공합니다.

### Task: Plan

In [23]:
plan = Task(
    description=(
        "1. {topic} 에 대한 최신 트렌드, 주요 플레이어,   "
            "주요 뉴스를 우선적으로 다룬다.\n"
        "2. 대상 독자를 파악하고, 그들의 관심사와 주요 문제점을 분석한다.\n"
        "3. 소개, 핵심 내용, 그리고 콜투액션(CTA)을 포함한 "
            "상세한 콘텐츠 개요를 작성한다.\n"
        "4. SEO 키워드와 관련 데이터 또는 출처를 포함한다."
    ),
    expected_output="콘텐츠 개요, 대상 독자 분석, "
        "SEO 키워드 및 참고 자료를 포함한 "
        "포괄적인 콘텐츠 계획 문서.",
    agent=planner,
)

### Task: Write

In [24]:
write = Task(
    description=(
        "1. 콘텐츠 기획을 활용하여 {topic} 에 대한 "
            "매력적인 블로그 게시글을 작성하세요.\n"
        "2. SEO 키워드를 자연스럽게 포함하세요.\n"
        "3. 섹션 및 부제목을 흥미롭게 구성하세요.\n"
        "4. 글의 구조는 흥미로운 도입부, 통찰력 있는 본문, "
            "요약하는 결론으로 구성하세요.\n"
        "5. 문법 오류를 수정하고 브랜드의 톤과 "
            "일관성을 유지하세요.\n"
    ),
    expected_output="마크다운 형식으로 작성된 완성도 높은 "
        "한글로 작성된 블로그 게시글. 각 섹션은 2~3개의 단락으로 구성되어야 합니다.",
    agent=writer,
)

### Task: Edit

In [25]:
edit = Task(
    description=("주어진 블로그 게시물을 교정하여 "
                 "문법 오류를 수정하고 "
                 "브랜드 칼라에 맞추세요."),
    expected_output="출판 준비가 완료된, 마크다운 형식의 한글로 잘 작성된 블로그 게시물. "
                    "각 섹션은 2~3개의 단락으로 구성되어야 합니다.",
    agent=editor
)

## Crew 생성

- 에이전트 Crew를 만들고, 해당 에이전트가 수행할 작업을 전달합니다.
- **참고**: task는 순차적으로 수행됩니다.
- `verbose=2`를 사용하면 실행 로그를 볼 수 있습니다.

In [26]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit]
)

## Crew 실행

In [27]:
result = crew.kickoff(inputs={"topic": "AI 에이전트"})

# Agent: 콘텐츠 기획자
## Task: 1. AI 에이전트 에 대한 최신 트렌드, 주요 플레이어,   주요 뉴스를 우선적으로 다룬다.
2. 대상 독자를 파악하고, 그들의 관심사와 주요 문제점을 분석한다.
3. 소개, 핵심 내용, 그리고 콜투액션(CTA)을 포함한 상세한 콘텐츠 개요를 작성한다.
4. SEO 키워드와 관련 데이터 또는 출처를 포함한다.


# Agent: 콘텐츠 기획자
## Final Answer: 
**콘텐츠 개요: AI 에이전트의 최신 트렌드 및 주요 플레이어**

1. **서론**
   - AI 에이전트의 정의 및 중요성 소개
   - 현재 AI 기술의 발전이 삶에 미치는 영향

2. **최신 트렌드**
   - **자연어 처리(NLP) 및 대화형 AI의 발전**
     - 예: OpenAI의 ChatGPT, Google의 Bard
   - **개인화 및 사용자 경험**
     - AI 에이전트를 통한 맞춤형 서비스 제공
   - **멀티모달 AI**
     - 텍스트, 이미지, 음성을 통합한 다양한 데이터 처리 방식
   - **AI 안전 및 윤리적 문제**
     - AI 에이전트의 투명성과 책임성에 대한 논의
   - **AI와 자동화의 결합**
     - 산업별 AI 솔루션 통합 사례

3. **주요 플레이어**
   - **OpenAI**: ChatGPT 및 API 서비스
   - **Google**: Bard 및 Cloud AI 플랫폼
   - **Amazon**: Alexa 및 AI 기반 서비스
   - **Microsoft**: Azure AI 솔루션 및 ChatGPT와의 파트너십
   - **IBM**: Watson AI 및 산업별 솔루션

4. **주요 뉴스**
   - 최신 AI 에이전트의 성공 사례 및 배포 소식
   - AI 기술규제에 대한 정책 변화
   - 주요 기업의 AI 투자 및 인수합병 소식

5. **타겟 독자 분석**
   - **대상 독자**: IT 전문가, 중소기업 경영자, 마케팅 전문

- 실행 결과를 노트북에 마크다운으로 표시합니다.

In [28]:
from IPython.display import Markdown
Markdown(result.raw)

# AI 에이전트의 최신 트렌드 및 주요 플레이어

## 서론

AI 에이전트는 인공지능 기술을 활용해 사용자와 상호작용하며 다양한 서비스를 제공하는 소프트웨어 시스템입니다. 이러한 AI 에이전트는 대화형 AI, 자동화 도구 및 개인화된 서비스 형태로 우리의 생활에 깊숙이 자리 잡고 있습니다. 현재 AI 기술의 발전은 단순한 정보 검색을 넘어 사용자의 요구에 맞춰 서비스를 조정하고 새로운 경험을 제공하는 데 중점을 두고 있습니다. 이러한 변화는 비즈니스 환경뿐만 아니라 개인의 일상생활에도 혁신을 가져오고 있습니다.

AI 에이전트의 발전은 기업의 생산성을 높이고 고객 경험을 개선하며, 나아가 경쟁력을 확보하는 데 크게 기여하고 있습니다. 따라서 AI 에이전트의 최신 동향을 파악하는 것은 더욱 중요해졌습니다.

## 최신 트렌드

### 자연어 처리(NLP) 및 대화형 AI의 발전

최근 자연어 처리(NLP)의 비약적인 발전으로 대화형 AI 기술이 크게 향상되었습니다. OpenAI의 ChatGPT와 Google의 Bard는 사용자가 자연어로 질문하였을 때 보다 정확하고 유창하게 응답할 수 있도록 설계되었습니다. 이들 AI 시스템은 머신러닝 알고리즘을 통해 사용자와의 대화에서도 지능적인 반응을 제공하며, 이는 비즈니스 지원 및 고객 서비스를 혁신적으로 변화시키고 있습니다.

### 개인화 및 사용자 경험

AI 에이전트는 개인화 서비스를 통해 사용자 경험을 한층 더 향상시키고 있습니다. 사용자 데이터를 바탕으로 맞춤형 정보를 제공하거나 특정 요구를 예측해 효율성을 높이는 것이 가능합니다. 이러한 개인화 기술은 고객 만족도를 높이며, 기업의 반복 거래를 유도하는 데 중요한 역할을 합니다.

### 멀티모달 AI

멀티모달 AI는 텍스트, 이미지, 음성 데이터를 통합하여 다양한 정보를 처리하는 시스템을 의미합니다. 이러한 AI는 사용자가 제공하는 다양한 형태의 데이터를 통해 보다 정교하고 효율적인 서비스 제공이 가능합니다. 예를 들어, 사용자가 별도의 설명 없이 음성으로 문의하면 AI는 텍스트나 이미지를 기반으로 추가 정보를 제공할 수 있습니다. 이러한 멀티모달 처리 기술은 여러 분야에서의 응용 가능성을 대폭 확장합니다.

### AI 안전 및 윤리적 문제

AI 에이전트의 발전과 함께 AI 안전 및 윤리적 문제에 대한 논의도 중요해지고 있습니다. AI의 투명성과 책임성에 관한 문제는 기업과 사회 전반에 걸쳐 큰 영향을 미칩니다. 예를 들어, AI 시스템이 잘못된 정보를 제공하거나 편향된 데이터를 학습할 경우 그 여파는 상당할 수 있기 때문에 이러한 윤리적 딜레마를 해결하기 위한 규정이 필요합니다. 이는 AI 에이전트의 신뢰성을 높이는 데 필수적입니다.

### AI와 자동화의 결합

AI 기술은 자동화와 결합되어 효율성을 한층 높이고 있습니다. 다양한 산업 분야에서 AI 솔루션이 도입되어 반복적인 업무를 자동화하고, 데이터 분석을 통해 인사이트를 도출하고 있습니다. 이러한 혼합 솔루션은 기업의 생산성 향상뿐만 아니라 인적 자원 관리의 혁신을 가져와 경영 전략에 긍정적인 영향을 미칩니다.

## 주요 플레이어

### OpenAI

OpenAI는 ChatGPT 및 API 서비스를 통해 대화형 AI 기술을 선도하고 있습니다. 이 회사는 자연어 처리 및 머신러닝에서 높은 성능을 자랑하며, 많은 기업들이 OpenAI의 기술을 활용해 고객 대응 및 내부 업무 프로세스를 개선하고 있습니다.

### Google

Google은 Bard 및 Cloud AI 플랫폼을 통해 AI 기술을 각종 산업에 적용하고 있습니다. Google의 AI 기술은 크고 다양한 사용자를 대상으로 개인화된 서비스를 제공하는 데 강점을 보이며, 데이터를 활용한 인사이트 추출이 뛰어납니다.

### Amazon

Amazon의 Alexa 및 AI 기반 서비스는 사용자와의 상호작용을 통해 다양한 기능을 수행하며, 가정의 스마트 환경을 조성하는 데 크게 기여하고 있습니다. 이 시스템은 소비자가 도움이 필요할 때 즉각적으로 반응함으로써 사용자 경험을 향상시키고 있습니다.

### Microsoft

Microsoft는 Azure AI 솔루션과 ChatGPT와의 파트너십을 통해 AI 기술의 활용 범위를 넓히고 있습니다. 이 플랫폼은 비즈니스 맞춤형 AI 솔루션을 제공하여 여러 산업 분야의 고객들에게 실질적인 가치를 제공하고 있습니다.

### IBM

IBM은 Watson AI를 통해 산업별 맞춤형 솔루션을 제공하고 있습니다. Watson은 의료, 금융, 제조업 등 다양한 분야에서 AI 기술을 통합하여 혁신적인 결과를 도출하고 있으며, AI의 전문성을 넓히고 있습니다.

## 주요 뉴스

최근 AI 에이전트의 성공 사례와 배포 소식이 잇따르고 있습니다. 특히 기업들이 AI 기술을 적극적으로 도입하고 있는 가운데, 새로운 AI 기술 규제에 대한 정책 변화도 주목할 만합니다. 또한 주요 기업들이 AI에 대한 투자를 지속적으로 확대하고 있음을 알 수 있습니다. 이러한 변화들은 AI 에이전트의 발전 방향과 시장 경쟁력에 큰 영향을 미칠 것입니다.

## 결론 및 콜 투 액션 (CTA)

AI 에이전트의 도입은 우리 생활과 비즈니스 환경을 획기적으로 변화시킬 것으로 기대됩니다. 맞춤형 서비스와 효율성을 더한 AI 에이전트는 경쟁력을 높이고 혁신적인 변화를 가져올 것입니다. “지금 바로 AI 에이전트를 통해 귀사의 경쟁력을 강화하세요!”라는 메시지를 통해 독자 여러분의 적극적인 참여를 유도합니다. AI 에이전트의 새로운 가능성을 함께 탐험해 보세요!

## 직접 시도해보세요

- 원하는 주제를 제출하고 에이전트가 무엇을 내놓는지 보세요!

In [29]:
topic = "온톨로지 기반 공간 AI 서비스"
result = crew.kickoff(inputs={"topic": topic})

# Agent: 콘텐츠 기획자
## Task: 1. 온톨로지 기반 공간 AI 서비스 에 대한 최신 트렌드, 주요 플레이어,   주요 뉴스를 우선적으로 다룬다.
2. 대상 독자를 파악하고, 그들의 관심사와 주요 문제점을 분석한다.
3. 소개, 핵심 내용, 그리고 콜투액션(CTA)을 포함한 상세한 콘텐츠 개요를 작성한다.
4. SEO 키워드와 관련 데이터 또는 출처를 포함한다.


# Agent: 콘텐츠 기획자
## Final Answer: 
### 콘텐츠 개요: 온톨로지 기반 공간 AI 서비스

#### 1. 서론
- 현대 사회의 디지털화를 이끄는 온톨로지 기반 공간 AI 서비스의 정의와 중요성
- 이 서비스들이 어떻게 공간 인지 및 사용자 경험을 개선하는지에 대한 간략한 개요.

#### 2. 최신 트렌드
- 온톨로지와 AI 결합의 발전: AI가 온톨로지를 어떻게 활용하여 공간 인식을 개선하고 있는지에 대한 설명.
- 실시간 데이터 분석 및 알고리즘 개선의 사례 연구.
- 주요 분야(스마트 시티, 건축, 물류)에 대한 전반적인 영향.

#### 3. 주요 플레이어
- Google, IBM, Microsoft 등 주요 기업이 온톨로지 기반 AI 서비스에서 어떤 역할을 하는지 구체적으로 기술.
- 혁신적인 스타트업 및 신생 기업 소개: 공간 기반 서비스를 제공하는 핀테크, 헬스케어 기업 등의 사례.
  
#### 4. 주요 뉴스
- 최근 6개월 내 온톨로지 기반 공간 AI 서비스 관련 주요 뉴스 및 발표 사항 정리.
- noteworthy 사건 및 인수합병 소식: 기술 발전과 시장 동향을 반영.

#### 5. 대상 독자 분석
- **대상:**
  - 기업 IT 의사결정자들, 공간 데이터 분석가, 스마트 시티 개발자, 그리고 인공지능 기술에 관심 있는 기술자들.
  
- **관심사 및 문제점:**
  - 신속한 데이터 처리와 AI의 활용 가능성.
  - 사용자 경험을 향상시키기 위한 효율적인 시스템 구축.
  - 비용 효율성 및 데이터 보안 문제.
  
#### 

In [30]:
Markdown(result.raw)

# 온톨로지 기반 공간 AI 서비스: 디지털 혁신의 새로운 패러다임

## 서론
현대 사회는 디지털화의 물결 속에 있으며, 이러한 발전의 중심에는 온톨로지 기반 공간 AI 서비스가 자리 잡고 있습니다. 이러한 서비스는 사용자 경험을 획기적으로 개선하고, 공간 인지에 대한 이해를 심화시키며, 기업의 의사결정을 지원하는 중요한 역할을 하고 있습니다. 예를 들어, 온톨로지 기반 기술은 위치 기반 서비스(LBS)와 결합되어, 사용자에게 맞춤형 정보를 제공하여 보다 효율적인 솔루션을 가능하게 합니다.

온톨로지 기반 공간 AI 서비스는 고객의 다양한 요구를 충족시키기 위해 데이터 분석과 공간 인식의 경계를 허물고 있습니다. 이를 통해 더욱 향상된 사용자 경험과 효율성을 제공하는 서비스가 실현되고 있습니다. 본 블로그에서는 이러한 서비스의 최신 트렌드, 주요 기업과 스타트업, 그리고 최근의 주요 뉴스를 살펴보겠습니다.

## 최신 트렌드
온톨로지와 인공지능(AI)의 결합은 전례 없는 발전을 이루고 있습니다. AI 기술은 온톨로지를 활용하여 공간 인식을 개선하고, 실시간 데이터 분석을 통해 공간의 맥락을 이해하고 사용자에게 맞춤형 서비스를 제공하도록 계속 진화하고 있습니다. 이러한 기술적 진보는 다양한 산업에서 신속하고 정확한 데이터 처리를 가능하게 합니다.

스마트 시티 개발의 경우, 온톨로지 기반 AI 서비스는 도시의 다양한 데이터(교통, 에너지 소비 등)를 실시간으로 분석하여 도시 관리의 효율성을 높이는 데 기여하고 있습니다. 또한, 물류 분야에서도 이러한 기술이 물품의 위치와 이동 경로를 최적화하여 운영 비용을 절감하는 데 큰 도움을 주고 있는 상황입니다.

## 주요 플레이어
온톨로지 기반 공간 AI 서비스 시장에서 Google, IBM, Microsoft와 같은 대기업들은 선도적인 역할을 수행하고 있습니다. 이들 기업은 강력한 인프라와 AI 기술을 바탕으로 시장에서의 경쟁력을 더욱 강화하고 있으며, 혁신적인 솔루션을 통해 고객의 다양한 니즈를 충족시키고 있습니다. 

또한, 최근에는 핀테크, 헬스케어 등 다양한 분야에서 공간 기반 서비스를 제공하는 혁신적인 스타트업과 신생 기업들이 등장하고 있습니다. 이들은 기존 산업의 패러다임을 변화시키며 새로운 시장을 창출하여 경쟁 우위를 점하고 있습니다.

## 주요 뉴스
최근 6개월간 온톨로지 기반 공간 AI 서비스와 관련된 주요 뉴스가 보도되었습니다. 여러 기업에서 이 기술을 활용한 혁신적인 솔루션을 출시하며 시장의 관심을 끌고 있으며, 인수합병과 같은 사건도 활성화되고 있습니다. 예를 들어, A사는 B사의 데이터 분석 기술을 인수하여 공간 AI 서비스의 성능을 한층 강화했다는 소식이 전해졌습니다.

이러한 발전은 기술 혁신과 함께 시장 동향을 반영하고 있으며, 온톨로지 기반 AI 서비스의 미래에 대한 기대감을 더욱 높이고 있습니다. 기업들은 이러한 변화에 발맞추어 대응 전략을 마련해야 할 것입니다.

## 대상 독자 분석
온톨로지 기반 공간 AI 서비스의 주요 대상 독자는 기업 IT 의사결정자, 공간 데이터 분석가, 스마트 시티 개발자 및 인공지능 기술에 관심 있는 기술자들입니다. 이들은 신속한 데이터 처리와 AI의 활용 가능성을 통해 사용자 경험을 개선하고, 효율적인 시스템 구축을 원하고 있습니다. 

또한, 비용 효율성과 데이터 보안 문제 또한 이들의 주요 관심사입니다. 이러한 배경 속에서 이들은 온톨로지 기반 공간 AI 서비스를 도입함으로써 경쟁력을 유지하고 비즈니스 기회를 극대화하려고 노력하고 있습니다.

## 결론
온톨로지 기반 공간 AI 서비스는 현대 사회에서 필수불가결한 요소로 자리 잡고 있습니다. 이 서비스는 공간 인식 능력을 획기적으로 개선하여 기업의 경쟁력을 높이는 중요한 역할을 하게 될 것입니다. 

이제 기업들은 더욱 창의적이고 효율적인 방식으로 데이터를 활용하여 사용자 경험을 극대화할 수 있는 기회를 찾아야 할 시점입니다. 이러한 혁신적인 서비스에 투자하는 것은 선택이 아닌 필수입니다.

지금 바로 온톨로지 기반 공간 AI 서비스의 데모를 신청해보세요. 미래의 기술을 경험하며 귀사의 비즈니스를 한 단계 더 발전시키는 기회를 놓치지 마시기 바랍니다. 또한 블로그를 구독하여 최신 동향을 지속적으로 확인하고, 변화에 민감하게 대응해 나가세요!